# ２章　小売店のデータでデータ加工を行う１０本ノック

本章では、ある小売店の売上履歴と顧客台帳データを用いて、データ分析の素地となる「データの加工」を習得することが目的です。
実際の現場データは手入力のExcel等、決して綺麗なデータではない事が多いため、
データの揺れや整合性の担保など、汚いデータを取り扱うデータ加工を主体に進めて行きます。

### ノック１１：データを読み込んでみよう

In [21]:
import pandas as pd

class Data:
    def __init__(self):
        self.uriage = pd.read_csv('uriage.csv')
        self.kokyaku_daicho = pd.read_excel('kokyaku_daicho.xlsx')

In [22]:
data = Data()
data.uriage.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二


### ノック１２：データの揺れを見てみよう

In [23]:
data.uriage['item_name'].head()

0      商品A
1    商 品 S
2    商 品 a
3      商品Z
4      商品a
Name: item_name, dtype: object

In [24]:
data.uriage['item_price'].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

### ノック１３：データに揺れがあるまま集計しよう

In [25]:
piv_dat = data.uriage.pivot_table(index='item_name', columns='customer_name', values='item_price')
piv_dat.head()

customer_name,さだ千佳子,中仁晶,中田美智子,丸山光臣,久保田倫子,亀井一徳,五十嵐春樹,井上桃子,井口寛治,井川真悠子,...,香椎優一,高原充則,高梨結衣,高沢美咲,高田さんま,鳥居広司,鶴岡薫,麻生莉緒,黄川田博之,黒谷長利
item_name,,,,,,,,,,,,,,,,,,,,,
商品W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
商 品 n,NaN,NaN,NaN,NaN,NaN,NaN,1400.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
商品E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
商品M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
商品P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### ノック１４：商品名の揺れを補正しよう

In [26]:
data.uriage['item_name'] = data.uriage['item_name'].str.replace(' ', '')
data.uriage['item_name'] = data.uriage['item_name'].str.upper()
data.uriage['item_name'].unique()

array(['商品A', '商品S', '商品Z', '商品V', '商品O', '商品U', '商品L', '商品C', '商品I',
       '商品R', '商品X', '商品G', '商品P', '商品Q', '商品Y', '商品N', '商品W', '商品E',
       '商品K', '商品B', '商品F', '商品D', '商品M', '商品H', '商品T', '商品J'],
      dtype=object)

### ノック１５：金額欠損値の補完をしよう

In [27]:
data.uriage.isnull().sum()

purchase_date      0
item_name          0
item_price       387
customer_name      0
dtype: int64

In [28]:
def fill_null(uriage: pd.DataFrame):
    price_of = {}
    for item_name, item_price in zip(uriage['item_name'], uriage['item_price']):
        if pd.isna(item_price):
            continue
        price_of[item_name] = item_price

    uriage['item_price'] = uriage['item_name'].map(price_of)


fill_null(data.uriage)
data.uriage.isnull().sum()

purchase_date    0
item_name        0
item_price       0
customer_name    0
dtype: int64

### ノック１６：顧客名の揺れを補正しよう

In [29]:
print(data.kokyaku_daicho['顧客名'].unique()[:50])
print(data.uriage['customer_name'].unique()[:50])

['須賀ひとみ' '岡田\u3000 敏也' '芳賀 希' '荻野  愛' '栗田 憲一' '梅沢 麻緒' '相原 ひとり' '新村 丈史'
 '石川  まさみ' '小栗 正義' '大倉 晃司' '那須 蒼甫' '清水 裕次郎' '榊原 しぼり' '高沢   美咲' '川島 友以乃'
 '唐沢 景子' '稲田 将也' '秋葉 あき' '西脇 礼子' '内村 まさみ' '進藤 瞬' '小口 豊' '笹原 しぼり' '岩佐 孝太郎'
 '河村 由樹' '菅原 誠治' '榎本 薫' '高梨 結衣' '鶴岡 薫' '藤木 一恵' '森岡 季衣' '田畑 正敏' '宇野 秀樹'
 '高原 充則' '梅村 秀樹' '浅田 賢二' '石渡 小雁' '吉岡 サダヲ' '井口 寛治' '奥 光洋' '佐藤 慶二' '東 光博'
 '高田 さんま' '合田 光' '深田 信輔' '島 英嗣' '梅津 淳子' '日比野 徹' '熊倉 綾']
['深井菜々美' '浅田賢二' '南部慶二' '麻生莉緒' '平田鉄二' '堀江佑' '深井照生' '牧田玲那' '堀北雅彦' '大地礼子'
 '矢部惇' '岡田敏也' '浅見広司' '熊井憲史' '黄川田博之' '尾形小雁' '神原美嘉' '若杉徹' '石渡小雁' '東光博'
 '日野夏希' '黒谷長利' '田辺光洋' '戸塚美幸' '榊原しぼり' '明石家明' '手塚進' '荻野愛' '菅原誠治' '石田佑'
 '片瀬長利' '岡慶太' '鈴木一哉' '小平陽子' '原口俊二' '井川真悠子' '金森なつみ' '本多フミヤ' '唐沢涼' '中田美智子'
 '松谷愛子' '深沢ひろ子' '五十嵐春樹' '白井俊二' '芦田博之' '楠高史' '石崎幸子' '本橋直人' '杉下悟志' '佐藤慶二']


In [30]:
data.kokyaku_daicho['顧客名'] = data.kokyaku_daicho['顧客名'].str.replace(' ', '')
data.kokyaku_daicho['顧客名'] = data.kokyaku_daicho['顧客名'].str.replace('　', '')

In [31]:
print(data.kokyaku_daicho['顧客名'].unique()[:50])
print(data.uriage['customer_name'].unique()[:50])

['須賀ひとみ' '岡田敏也' '芳賀希' '荻野愛' '栗田憲一' '梅沢麻緒' '相原ひとり' '新村丈史' '石川まさみ' '小栗正義'
 '大倉晃司' '那須蒼甫' '清水裕次郎' '榊原しぼり' '高沢美咲' '川島友以乃' '唐沢景子' '稲田将也' '秋葉あき' '西脇礼子'
 '内村まさみ' '進藤瞬' '小口豊' '笹原しぼり' '岩佐孝太郎' '河村由樹' '菅原誠治' '榎本薫' '高梨結衣' '鶴岡薫'
 '藤木一恵' '森岡季衣' '田畑正敏' '宇野秀樹' '高原充則' '梅村秀樹' '浅田賢二' '石渡小雁' '吉岡サダヲ' '井口寛治'
 '奥光洋' '佐藤慶二' '東光博' '高田さんま' '合田光' '深田信輔' '島英嗣' '梅津淳子' '日比野徹' '熊倉綾']
['深井菜々美' '浅田賢二' '南部慶二' '麻生莉緒' '平田鉄二' '堀江佑' '深井照生' '牧田玲那' '堀北雅彦' '大地礼子'
 '矢部惇' '岡田敏也' '浅見広司' '熊井憲史' '黄川田博之' '尾形小雁' '神原美嘉' '若杉徹' '石渡小雁' '東光博'
 '日野夏希' '黒谷長利' '田辺光洋' '戸塚美幸' '榊原しぼり' '明石家明' '手塚進' '荻野愛' '菅原誠治' '石田佑'
 '片瀬長利' '岡慶太' '鈴木一哉' '小平陽子' '原口俊二' '井川真悠子' '金森なつみ' '本多フミヤ' '唐沢涼' '中田美智子'
 '松谷愛子' '深沢ひろ子' '五十嵐春樹' '白井俊二' '芦田博之' '楠高史' '石崎幸子' '本橋直人' '杉下悟志' '佐藤慶二']


### ノック１７：日付の揺れを補正しよう

In [33]:
display(data.kokyaku_daicho.head())
def fix_date():
    is_serial = data.kokyaku_daicho['登録日'].astype('str').str.isdigit()
    serial_date = pd.to_timedelta(data.kokyaku_daicho.loc[is_serial, '登録日'], unit='D') + pd.to_datetime('1900-1-1')
    non_serial_date = pd.to_datetime(data.kokyaku_daicho.loc[~is_serial, '登録日'])
    data.kokyaku_daicho.loc[is_serial, '登録日'] = serial_date
    data.kokyaku_daicho.loc[~is_serial, '登録日'] = non_serial_date


fix_date()
data.kokyaku_daicho.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04 00:00:00
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-18 00:00:00
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07 00:00:00
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-19 00:00:00
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-29 00:00:00


,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04 00:00:00
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-18 00:00:00
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07 00:00:00
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-19 00:00:00
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-29 00:00:00


顧客名                       object
かな                        object
地域                        object
メールアドレス                   object
登録日               datetime64[ns]
perchase_month            object
dtype: object

### ノック１８：顧客名をキーに２つのデータを結合(ジョイン)しよう

In [19]:
join_data = pd.merge(data.uriage, data.kokyaku_daicho, left_on='customer_name', right_on='顧客名', how='left')
join_data.drop('customer_name', axis=1, inplace=True)
join_data.head()

,purchase_date,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日,perchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26,201701
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07,201804
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19,201806
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22,201807
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07,201706


### ノック１９：クレンジングしたデータをダンプしよう

In [20]:
def columns_reorder(data: pd.DataFrame):
    column_list = ['purchase_date', 'perchase_month', 'item_name', 'item_price', '顧客名', 'かな', '地域', 'メールアドレス', '登録日']
    return data[column_list]

join_data = columns_reorder(join_data)
display(join_data.head())

,purchase_date,perchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日
0,2019-06-13 18:02:34,201701,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26
1,2019-07-13 13:05:29,201804,商品S,1900.0,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07
2,2019-05-11 19:42:07,201806,商品A,100.0,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19
3,2019-02-12 23:40:45,201807,商品Z,2600.0,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22
4,2019-04-22 03:09:35,201706,商品A,100.0,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07


In [43]:
join_data.to_csv('dump_data.csv', index=False)

### ノック２０：データを集計しよう

In [44]:
read_data = pd.read_csv('dump_data.csv')

In [45]:
piv_data = read_data.pivot_table(index='perchase_month',
                                 columns='item_name',
                                 values='item_price',
                                 aggfunc='sum')
piv_data

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
perchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1800.0,2600.0,5700.0,6800.0,9000.0,9000.0,7700.0,12800.0,16200.0,17000.0,...,28900.0,37800.0,38000.0,34000.0,14700.0,48400.0,29900.0,33600.0,25000.0,NaN
201902,1900.0,2800.0,7800.0,8400.0,8000.0,8400.0,9800.0,13600.0,10800.0,14000.0,...,37400.0,39600.0,41800.0,46000.0,39900.0,48400.0,55200.0,38400.0,27500.0,2600.0
201903,1700.0,4200.0,6000.0,6800.0,4500.0,16200.0,9800.0,14400.0,10800.0,16000.0,...,39100.0,28800.0,38000.0,24000.0,48300.0,39600.0,36800.0,50400.0,40000.0,NaN
201904,1700.0,3800.0,7200.0,8000.0,9000.0,10200.0,9800.0,8800.0,16200.0,13000.0,...,34000.0,36000.0,30400.0,32000.0,23100.0,33000.0,32200.0,38400.0,50000.0,NaN
201905,2400.0,2800.0,4800.0,5600.0,9500.0,10800.0,16100.0,12000.0,14400.0,11000.0,...,22100.0,39600.0,34200.0,32000.0,33600.0,19800.0,48300.0,38400.0,50000.0,NaN
201906,2400.0,2400.0,3300.0,7600.0,6500.0,10800.0,10500.0,10400.0,17100.0,22000.0,...,25500.0,28800.0,39900.0,24000.0,37800.0,44000.0,39100.0,36000.0,32500.0,NaN
201907,2000.0,4000.0,5100.0,6800.0,6000.0,10200.0,13300.0,15200.0,17100.0,23000.0,...,25500.0,34200.0,43700.0,42000.0,27300.0,61600.0,36800.0,43200.0,30000.0,NaN


In [46]:
piv_data = read_data.pivot_table(index='item_name',
                                 columns='perchase_month',
                                 values='item_price',
                                 aggfunc='sum')
piv_data

perchase_month,201901,201902,201903,201904,201905,201906,201907
item_name,,,,,,,
商品A,1800.0,1900.0,1700.0,1700.0,2400.0,2400.0,2000.0
商品B,2600.0,2800.0,4200.0,3800.0,2800.0,2400.0,4000.0
商品C,5700.0,7800.0,6000.0,7200.0,4800.0,3300.0,5100.0
商品D,6800.0,8400.0,6800.0,8000.0,5600.0,7600.0,6800.0
商品E,9000.0,8000.0,4500.0,9000.0,9500.0,6500.0,6000.0
商品F,9000.0,8400.0,16200.0,10200.0,10800.0,10800.0,10200.0
商品G,7700.0,9800.0,9800.0,9800.0,16100.0,10500.0,13300.0
商品H,12800.0,13600.0,14400.0,8800.0,12000.0,10400.0,15200.0
商品I,16200.0,10800.0,10800.0,16200.0,14400.0,17100.0,17100.0


In [49]:
kokyaku = pd.merge(data.kokyaku_daicho, data.uriage, left_on='顧客名', right_on='customer_name', how='left')
kokyaku = kokyaku[kokyaku['item_name'].isnull()]
kokyaku

,顧客名,かな,地域,メールアドレス,登録日,purchase_date,item_name,item_price,customer_name,perchase_month
2999,福井美希,ふくい みき,D市,fukui_miki1@example.com,2019-04-23,NaT,NaN,NaN,NaN,NaN
